## DX799S O1 Data Science Capstone (Summer 1 2025): ACTIVITY 2.2 ##

Each week, you will apply the concepts of that week to your Integrated Capstone Project’s dataset. In preparation for Milestone One, create a Jupyter Notebook (similar to in Module B, semester two) that illustrates these lessons. There are no specific questions to answer in your Jupyter Notebook files in this course; your general goal is to analyze your data, using the methods you have learned about in this course and in this program, and draw interesting conclusions. 

For Week 2, include concepts such as linear regression with lasso, ridge, and elastic net regression. This homework will be submitted for peer review and feedback in Week 3 in the assignment titled 3.4 Peer Review: Week 2 Jupyter Notebook. Complete your Jupyter Notebook homework by 11:59 pm ET on Sunday.

In Week 7, you will compile your findings from your Jupyter Notebook homework into your Milestone One assignment for grading. For full instructions and the rubric for Milestone One, refer to the following link. 

In [100]:
# ===================================
# Useful Imports: Add more as needed
# ===================================

# Standard Libraries
import os
import time
import math
import io
import zipfile
import requests
from urllib.parse import urlparse
from itertools import chain, combinations

# Data Science Libraries
import numpy as np
import pandas as pd
import seaborn as sns

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as mticker  # Optional: Format y-axis labels as dollars
import seaborn as sns

# Scikit-learn (Machine Learning)
from sklearn.model_selection import (
    train_test_split, 
    cross_val_score, 
    GridSearchCV, 
    RandomizedSearchCV, 
    RepeatedKFold
)
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector, f_regression, SelectKBest
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder




In [101]:
repeated_cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)

In [102]:
#Video Review Dataset 

df_videoreview = pd.read_csv("video_review.csv")

label_encoder = LabelEncoder()

print("Object columns before encoding:")
print(df_videoreview.select_dtypes(include=['object']).columns)

for col in df_videoreview.select_dtypes(include=['object']).columns:
    df_videoreview[col] = label_encoder.fit_transform(df_videoreview[col].astype(str))


target_column = 'Primary_Impact_Type'  
X = df_videoreview.drop(columns=[target_column])
y = df_videoreview[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



Object columns before encoding:
Index(['Player_Activity_Derived', 'Turnover_Related', 'Primary_Impact_Type',
       'Primary_Partner_GSISID', 'Primary_Partner_Activity_Derived',
       'Friendly_Fire'],
      dtype='object')


In [103]:
# Video Review with Lasso Regression using Primary_Impact_Type as Target

lasso = Lasso(alpha=1.0, max_iter=10000)

neg_mse_scores = cross_val_score(lasso, X_train_scaled, y_train, cv=repeated_cv, scoring='neg_mean_squared_error')
mse_scores = -neg_mse_scores

rmse_scores = np.sqrt(mse_scores)

mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)

print(f'Mean CV RMSE for Lasso Regression on Video Review Dataset: {mean_rmse:,.2f}')
print(f'Standard Deviation CV RMSE for Lasso Regression on Video Review Dataset: {std_rmse:,.2f}')


Mean CV RMSE for Lasso Regression on Video Review Dataset: 1.07
Standard Deviation CV RMSE for Lasso Regression on Video Review Dataset: 0.11


In [104]:
# Video Review with Ridge Regression using Primary_Impact_Type as Target

ridge = Ridge()

neg_mse_scores = cross_val_score(ridge, X_train_scaled, y_train, cv=repeated_cv, scoring='neg_mean_squared_error')
mse_scores = -neg_mse_scores

rmse_scores = np.sqrt(mse_scores)

mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)

print(f'Mean CV RMSE for Ridge Regression on Video Review Dataset: ${mean_rmse:,.2f}')
print(f'Standard Deviation CV RMSE for Ridge Regression on Video Review Dataset: {std_rmse:,.2f}')

Mean CV RMSE for Ridge Regression on Video Review Dataset: $1.25
Standard Deviation CV RMSE for Ridge Regression on Video Review Dataset: 0.30


In [105]:
# Video Review with Elastic Net Regression using Primary_Impact_Type as Target Tuning Parameters

param_grid = {
    'alpha': np.logspace(-3, 1, 10),        
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]  
}

elastic_net = ElasticNet(max_iter=10000)

grid_search = GridSearchCV(
    estimator=elastic_net,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=repeated_cv,  
    n_jobs=-1        
)

grid_search.fit(X_train_scaled, y_train)

best_params = grid_search.best_params_
best_rmse = np.sqrt(-grid_search.best_score_)

print(f"Best alpha: {best_params['alpha']}")
print(f"Best l1_ratio: {best_params['l1_ratio']}")
print(f"Best cross-validated RMSE: {best_rmse:.2f}")



Best alpha: 1.2915496650148828
Best l1_ratio: 0.5
Best cross-validated RMSE: 1.07


In [106]:
# Video Review with Elastic Net Regression using Primary_Impact_Type as Target

elastic_net = ElasticNet(alpha=1.2915496650148828, l1_ratio=0.5, max_iter=10000)

neg_mse_scores = cross_val_score(
    elastic_net,
    X_train_scaled,
    y_train,
    cv=repeated_cv,
    scoring='neg_mean_squared_error'
)

mse_scores = -neg_mse_scores
rmse_scores = np.sqrt(mse_scores)

mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)

print(f'Mean CV RMSE for Elastic Net on Video Review Dataset: {mean_rmse:,.2f}')
print(f'Standard Deviation CV RMSE on Video Review Dataset: {std_rmse:,.2f}')


Mean CV RMSE for Elastic Net on Video Review Dataset: 1.07
Standard Deviation CV RMSE on Video Review Dataset: 0.11


In [107]:
df_injuryrecord = pd.read_csv("InjuryRecord.csv")

In [108]:
label_encoder = LabelEncoder()

print("Object columns before encoding:")
print(df_injuryrecord.select_dtypes(include=['object']).columns)

for col in df_injuryrecord.select_dtypes(include=['object']).columns:
    df_injuryrecord[col] = label_encoder.fit_transform(df_injuryrecord[col].astype(str))


target_column = 'Surface'  
X_injury = df_injuryrecord.drop(columns=[target_column])
y_injury = df_injuryrecord[target_column]

X_train, X_test, y_train, y_test = train_test_split(X_injury, y_injury, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Object columns before encoding:
Index(['GameID', 'PlayKey', 'BodyPart', 'Surface'], dtype='object')


In [109]:
# Injury Record with Lasso Regression using Surface Type as Target

lasso = Lasso(alpha=1.0, max_iter=10000)

neg_mse_scores = cross_val_score(lasso, X_train_scaled, y_train, cv=repeated_cv, scoring='neg_mean_squared_error')
mse_scores = -neg_mse_scores

rmse_scores = np.sqrt(mse_scores)

mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)

print(f'Mean CV RMSE for Lasso Regression on Injury Record Dataset: {mean_rmse:,.2f}')
print(f'Standard Deviation CV RMSE for Lasso Regression on Injury Record Dataset: {std_rmse:,.2f}')

Mean CV RMSE for Lasso Regression on Injury Record Dataset: 0.51
Standard Deviation CV RMSE for Lasso Regression on Injury Record Dataset: 0.01


In [110]:
# Injury Record with Ridge Regression using Surface Type as Target

ridge = Ridge()

neg_mse_scores = cross_val_score(ridge, X_train_scaled, y_train, cv=repeated_cv, scoring='neg_mean_squared_error')
mse_scores = -neg_mse_scores

rmse_scores = np.sqrt(mse_scores)

mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)

print(f'Mean CV RMSE for Ridge Regression on Injury Record dataset: {mean_rmse:,.2f}')
print(f'Standard Deviation CV RMSE for Ridge Regression Injury Record dataset: {std_rmse:,.2f}')


Mean CV RMSE for Ridge Regression on Injury Record dataset: 0.54
Standard Deviation CV RMSE for Ridge Regression Injury Record dataset: 0.04


In [118]:
# Injury Record with Elastic Net Regression using Surface Type as Target Tuning Parameters

param_grid = {
    'alpha': np.logspace(-3, 1, 10),        
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]  
}

elastic_net = ElasticNet(max_iter=10000)

grid_search = GridSearchCV(
    estimator=elastic_net,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=repeated_cv,  
    n_jobs=-1        
)

grid_search.fit(X_train_scaled, y_train)

best_params = grid_search.best_params_
best_rmse = np.sqrt(-grid_search.best_score_)

print(f"Best alpha: {best_params['alpha']}")
print(f"Best l1_ratio: {best_params['l1_ratio']}")
print(f"Best cross-validated RMSE: {best_rmse:.2f}")


Best alpha: 0.1668100537200059
Best l1_ratio: 0.9
Best cross-validated RMSE: 0.51


In [119]:
# Injury Record with Elastic Net Regression using Surface Type as Target

elastic_net = ElasticNet(alpha=0.1668100537200059, l1_ratio=0.9, max_iter=10000)

neg_mse_scores = cross_val_score(
    elastic_net,
    X_train_scaled,
    y_train,
    cv=repeated_cv,
    scoring='neg_mean_squared_error'
)

mse_scores = -neg_mse_scores
rmse_scores = np.sqrt(mse_scores)

mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)

print(f'Mean CV RMSE for Elastic Net on Injury Record Dataset: {mean_rmse:,.2f}')
print(f'Standard Deviation CV RMSE on Injury Record Dataset: {std_rmse:,.2f}')


Mean CV RMSE for Elastic Net on Injury Record Dataset: 0.51
Standard Deviation CV RMSE on Injury Record Dataset: 0.01


In [124]:
df_playlist = pd.read_csv("PlayList.csv")

label_encoder = LabelEncoder()

print("Object columns before encoding:")
print(df_playlist.select_dtypes(include=['object']).columns)

for col in df_playlist.select_dtypes(include=['object']).columns:
    df_playlist[col] = label_encoder.fit_transform(df_playlist[col].astype(str))


target_column = 'PlayType'  
X_playlist = df_playlist.drop(columns=[target_column])
y_playlist = df_playlist[target_column]

X_train, X_test, y_train, y_test = train_test_split(X_playlist, y_playlist, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Object columns before encoding:
Index(['GameID', 'PlayKey', 'RosterPosition', 'StadiumType', 'FieldType',
       'Weather', 'PlayType', 'Position', 'PositionGroup'],
      dtype='object')


In [125]:
# Playlist with Lasso Regression using Play Type as Target

lasso = Lasso(alpha=1.0, max_iter=10000)

neg_mse_scores = cross_val_score(lasso, X_train_scaled, y_train, cv=repeated_cv, scoring='neg_mean_squared_error')
mse_scores = -neg_mse_scores

rmse_scores = np.sqrt(mse_scores)

mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)

print(f'Mean CV RMSE for Lasso Regression on PlayList Dataset: {mean_rmse:,.2f}')
print(f'Standard Deviation CV RMSE for Lasso Regression on PlayList Dataset: {std_rmse:,.2f}')

Mean CV RMSE for Lasso Regression on PlayList Dataset: 2.35
Standard Deviation CV RMSE for Lasso Regression on PlayList Dataset: 0.01


In [ ]:
# Playlist with Ridge Regression using Play Type as Target
ridge = Ridge()

neg_mse_scores = cross_val_score(ridge, X_train_scaled, y_train, cv=repeated_cv, scoring='neg_mean_squared_error')
mse_scores = -neg_mse_scores

rmse_scores = np.sqrt(mse_scores)

mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)

print(f'Mean CV RMSE for Ridge Regression on PlayList dataset: {mean_rmse:,.2f}')
print(f'Standard Deviation CV RMSE for Ridge Regression PlayList dataset: {std_rmse:,.2f}')


Mean CV RMSE for Ridge Regression on Injury Record dataset: 2.35
Standard Deviation CV RMSE for Ridge Regression Injury Record dataset: 0.01


In [127]:
# Playlist with Elastic Net Regression using Play Type as Target Tuning Parameters
param_grid = {
    'alpha': np.logspace(-3, 1, 10),        
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]  
}

elastic_net = ElasticNet(max_iter=10000)

grid_search = GridSearchCV(
    estimator=elastic_net,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=repeated_cv,  
    n_jobs=-1        
)

grid_search.fit(X_train_scaled, y_train)

best_params = grid_search.best_params_
best_rmse = np.sqrt(-grid_search.best_score_)

print(f"Best alpha: {best_params['alpha']}")
print(f"Best l1_ratio: {best_params['l1_ratio']}")
print(f"Best cross-validated RMSE: {best_rmse:.2f}")



Best alpha: 0.001
Best l1_ratio: 0.1
Best cross-validated RMSE: 2.35


In [128]:
# Playlist with Elastic Net Regression using Play Type as Target

elastic_net = ElasticNet(alpha=0.001, l1_ratio=0.1, max_iter=10000)

neg_mse_scores = cross_val_score(
    elastic_net,
    X_train_scaled,
    y_train,
    cv=repeated_cv,
    scoring='neg_mean_squared_error'
)

mse_scores = -neg_mse_scores
rmse_scores = np.sqrt(mse_scores)

mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)

print(f'Mean CV RMSE for Elastic Net on Injury Record Dataset: {mean_rmse:,.2f}')
print(f'Standard Deviation CV RMSE on Injury Record Dataset: {std_rmse:,.2f}')


Mean CV RMSE for Elastic Net on Injury Record Dataset: 2.35
Standard Deviation CV RMSE on Injury Record Dataset: 0.01
